In [1]:
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchvision-0.8.1+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchaudio-0.7.0-cp37-cp37m-linux_x86_64.whl' --no-deps

!mkdir -p /kaggle/tmp && cp /kaggle/input/pytorch-170-cuda-toolkit-110221/cudatoolkit-11.0.221-h6bb024c_0 /kaggle/tmp/cudatoolkit-11.0.221-h6bb024c_0.tar.bz2 && conda install /kaggle/tmp/cudatoolkit-11.0.221-h6bb024c_0.tar.bz2 -y --offline

!pip install '/kaggle/input/mmdetectionv2140/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmcv_full-1_3_8-cu110-torch1_7_0/mmcv_full-1.3.8-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/pycocotools-2.0.2/pycocotools-2.0.2' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/mmpycocotools-12.0.3/mmpycocotools-12.0.3' --no-deps

!cp -r /kaggle/input/mmdetectionv2140/mmdetection-2.14.0 /kaggle/working/
!mv /kaggle/working/mmdetection-2.14.0 /kaggle/working/mmdetection
%cd /kaggle/working/mmdetection
!pip install -e . --no-deps
%cd /kaggle/working/

Processing /kaggle/input/pytorch-170-cuda-toolkit-110221/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.7.0
    Uninstalling torch-1.7.0:
      Successfully uninstalled torch-1.7.0
Processing /kaggle/input/pytorch-170-cuda-toolkit-110221/torchvision-0.8.1+cu110-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.1
    Uninstalling torchvision-0.8.1:
      Successfully uninstalled torchvision-0.8.1
Processing /kaggle/input/pytorch-170-cuda-toolkit-110221/torchaudio-0.7.0-cp37-cp37m-linux_x86_64.whl
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.7.0a0+ac17b64
    Uninstalling torchaudio-0.7.0a0+ac17b64:
      Successfully uninstalled torchaudio-0.7.0a0+ac17b64

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction:

In [2]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import sys
import os
from PIL import Image
import cv2
import gc

In [3]:
df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')

In [4]:
filepaths = glob('/kaggle/input/siim-covid19-detection/test/**/*dcm',recursive=True)
test_df = pd.DataFrame(data={'filepath': filepaths})
test_df['image_id'] = test_df.filepath.map(lambda x: x.split('/')[-1].replace('.dcm', '')+'_image')
test_df['study_id'] = test_df.filepath.map(lambda x: x.split('/')[-3].replace('.dcm', '')+'_study')

In [5]:
!cp /kaggle/input/gdcm-conda-install/gdcm.tar .
!tar -xvzf gdcm.tar
!conda install --offline ./gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2
!rm -rf ./gdcm.tar

gdcm/
gdcm/conda-4.8.4-py37hc8dfbb8_2.tar.bz2
gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2
gdcm/libjpeg-turbo-2.0.3-h516909a_1.tar.bz2

######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - done


In [6]:
TEST_PATH = 'kaggle/tmp/test'
os.makedirs(TEST_PATH, exist_ok=True)

In [7]:
import pydicom 
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [8]:
def dicom2array(path, voi_lut=True, fix_monochrome=True, for_save=False):
    dicom = pydicom.read_file(path)
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    if for_save:
        return data
    image = cv2.cvtColor(data, cv2.COLOR_GRAY2RGB)
    return image

In [9]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    im = Image.fromarray(array)
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    return im

In [10]:
test_df[['original_W', 'original_H']] = None
test_df['image_path'] = None

In [11]:
for i in tqdm(range(test_df.shape[0])):
    filepath = test_df.loc[i, 'filepath']
    image = dicom2array(filepath, for_save=True)
    test_df.loc[i, ['original_W', 'original_H']] = image.shape[0], image.shape[1]
    test_df.loc[i, 'image_path'] = TEST_PATH+'/'+test_df.loc[i, 'image_id'] + '.png'
    image = resize(image, size=512)
    image.save(TEST_PATH+'/'+test_df.loc[i, 'image_id'] + '.png')

100%|██████████| 1263/1263 [09:17<00:00,  2.27it/s]


In [12]:
test_df

,filepath,image_id,study_id,original_W,original_H,image_path
0,/kaggle/input/siim-covid19-detection/test/2fb1...,a29c5a68b07b_image,2fb11712bc93_study,2320,2828,kaggle/tmp/test/a29c5a68b07b_image.png
1,/kaggle/input/siim-covid19-detection/test/19c6...,9850b5470fd6_image,19c66935e737_study,2330,2382,kaggle/tmp/test/9850b5470fd6_image.png
2,/kaggle/input/siim-covid19-detection/test/2fc5...,8d6dea06a032_image,2fc50bf199cd_study,2422,3344,kaggle/tmp/test/8d6dea06a032_image.png
3,/kaggle/input/siim-covid19-detection/test/97c5...,dfc5c09a50bc_image,97c5d6eb413d_study,1140,1387,kaggle/tmp/test/dfc5c09a50bc_image.png
4,/kaggle/input/siim-covid19-detection/test/593c...,7230234e120a_image,593c3f815635_study,2318,2383,kaggle/tmp/test/7230234e120a_image.png
...,...,...,...,...,...,...
1258,/kaggle/input/siim-covid19-detection/test/b12d...,a43200bd5ceb_image,b12d2e5c39b7_study,2336,2836,kaggle/tmp/test/a43200bd5ceb_image.png
1259,/kaggle/input/siim-covid19-detection/test/6484...,37bf83df1b86_image,6484393291ea_study,3480,4240,kaggle/tmp/test/37bf83df1b86_image.png
1260,/kaggle/input/siim-covid19-detection/test/6484...,b30d2aef985f_image,6484393291ea_study,3480,4240,kaggle/tmp/test/b30d2aef985f_image.png
1261,/kaggle/input/siim-covid19-detection/test/531a...,a37a362df0ac_image,531aa20ff7c3_study,3480,4240,kaggle/tmp/test/a37a362df0ac_image.png


In [13]:
import tensorflow as tf

In [14]:
def read_image(image_path, image_size=(512, 512)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.
    image = tf.image.resize(image, image_size)
    return image

In [15]:
def apply_augmentation(image):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image

In [16]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

In [17]:
def get_dataset(image_paths, image_size=512, TTA=False):
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    if image_size == 512:
        dataset = dataset.map(lambda im: read_image(im, image_size=(512, 512)), num_parallel_calls=AUTO)
    else:
        dataset = dataset.map(lambda im: read_image(im, image_size=(600, 600)), num_parallel_calls=AUTO)
    dataset = dataset.map(apply_augmentation, num_parallel_calls=AUTO) if TTA else dataset
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

In [18]:
sys.path.append('/kaggle/input/efficientnet-keras-dataset/efficientnet_kaggle')
import efficientnet.tfkeras as efn

In [19]:
def get_classification_model(input_shape, two_classes=False):
    num_outputs = 4
    if two_classes:
        num_outputs = 1
    model = tf.keras.Sequential([
            efn.EfficientNetB7(
                input_shape=(*input_shape, 3),
                weights='imagenet',
                include_top=False),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(num_outputs, activation='softmax')
        ])
    return model

In [20]:
test_dataset = get_dataset(test_df.image_path.tolist(), image_size=600)

In [21]:
model=get_classification_model(input_shape=(600, 600))

In [22]:
num_folds = 5
predict = []
for i in tqdm(range(num_folds)):
    model_path = f'../input/slim-weights-effnet/model{i}.h5'
    model.load_weights(model_path)
    pred = model.predict(test_dataset)
    predict.append(pred)
    del pred 
    gc.collect()
predictions = np.mean(predict, axis=0)

100%|██████████| 5/5 [05:22<00:00, 64.60s/it]


In [23]:
data = {'id': test_df.study_id.tolist()}
study_df = pd.DataFrame(data=data)
study_df['PredictionString'] = None
study_df[['negative', 'typical', 'indeterminate', 'atypical']] = predictions

In [24]:
study_df.head()

,id,PredictionString,negative,typical,indeterminate,atypical
0,2fb11712bc93_study,None,0.000980,0.955375,0.039328,0.004316
1,19c66935e737_study,None,0.021758,0.728365,0.198130,0.051747
2,2fc50bf199cd_study,None,0.022755,0.784237,0.174068,0.018940
3,97c5d6eb413d_study,None,0.034062,0.682686,0.213291,0.069961
4,593c3f815635_study,None,0.000524,0.716001,0.170541,0.112934


In [25]:
for i in tqdm(range(study_df.shape[0])):
    negative = study_df.loc[i, 'negative']
    typical = study_df.loc[i, 'typical']
    indeterminate = study_df.loc[i, 'indeterminate']
    atypical = study_df.loc[i, 'atypical']
    study_df.loc[i, 'PredictionString'] = f'negative {negative:.3f} typical {typical:.3f} indeterminate {indeterminate:.3f} atypical {atypical:.3f}'
    

100%|██████████| 1263/1263 [00:00<00:00, 2117.12it/s]


In [26]:
study_df.sort_values(by=['id'])

,id,PredictionString,negative,typical,indeterminate,atypical
497,00188a671292_study,negative 0.713 typical 0.089 indeterminate 0.1...,0.713201,0.089293,0.172996,0.024510
123,004bd59708be_study,negative 0.002 typical 0.870 indeterminate 0.1...,0.001976,0.869992,0.110589,0.017443
379,00508faccd39_study,negative 0.731 typical 0.110 indeterminate 0.1...,0.731285,0.109995,0.137310,0.021410
1115,006486aa80b2_study,negative 0.259 typical 0.265 indeterminate 0.3...,0.259218,0.264679,0.380191,0.095912
233,00655178fdfc_study,negative 0.544 typical 0.209 indeterminate 0.2...,0.544469,0.209278,0.216249,0.030004
...,...,...,...,...,...,...
140,ff1ba0e9aaf0_study,negative 0.193 typical 0.564 indeterminate 0.2...,0.193027,0.564032,0.216929,0.026013
131,ff2cc4de58c5_study,negative 0.022 typical 0.218 indeterminate 0.5...,0.021868,0.218203,0.560105,0.199825
1164,ff2f0a744930_study,negative 0.001 typical 0.952 indeterminate 0.0...,0.000574,0.952374,0.042225,0.004827
115,ff88940dce8b_study,negative 0.412 typical 0.274 indeterminate 0.2...,0.412274,0.274301,0.270850,0.042575


In [27]:
del model, test_dataset
gc.collect()

76803

In [28]:
#model=get_classification_model(input_shape=(512, 512), two_classes=True)

In [29]:
test_df['none'] = None

In [30]:
test_dataset = get_dataset(test_df.image_path.tolist(), image_size=512)

In [31]:
with tf.device('GPU:0'):
    models = []
    MODEL_PATH = '../input/siimcovid19efnb7trainfold052class/'
    model1 = tf.keras.models.load_model(MODEL_PATH + 'model0.h5')
    model2 = tf.keras.models.load_model(MODEL_PATH + 'model1.h5')
    model3 = tf.keras.models.load_model(MODEL_PATH + 'model2.h5')
    model4 = tf.keras.models.load_model(MODEL_PATH + 'model3.h5')
    model5 = tf.keras.models.load_model(MODEL_PATH + 'model4.h5')
    models.append(model1)
    models.append(model2)
    models.append(model3)
    models.append(model4)
    models.append(model5)

predictions = sum([model.predict(test_dataset) for model in models]) / 5

In [32]:
test_df['none'] = predictions

In [33]:
test_df

,filepath,image_id,study_id,original_W,original_H,image_path,none
0,/kaggle/input/siim-covid19-detection/test/2fb1...,a29c5a68b07b_image,2fb11712bc93_study,2320,2828,kaggle/tmp/test/a29c5a68b07b_image.png,0.018594
1,/kaggle/input/siim-covid19-detection/test/19c6...,9850b5470fd6_image,19c66935e737_study,2330,2382,kaggle/tmp/test/9850b5470fd6_image.png,0.082357
2,/kaggle/input/siim-covid19-detection/test/2fc5...,8d6dea06a032_image,2fc50bf199cd_study,2422,3344,kaggle/tmp/test/8d6dea06a032_image.png,0.103633
3,/kaggle/input/siim-covid19-detection/test/97c5...,dfc5c09a50bc_image,97c5d6eb413d_study,1140,1387,kaggle/tmp/test/dfc5c09a50bc_image.png,0.174045
4,/kaggle/input/siim-covid19-detection/test/593c...,7230234e120a_image,593c3f815635_study,2318,2383,kaggle/tmp/test/7230234e120a_image.png,0.035506
...,...,...,...,...,...,...,...
1258,/kaggle/input/siim-covid19-detection/test/b12d...,a43200bd5ceb_image,b12d2e5c39b7_study,2336,2836,kaggle/tmp/test/a43200bd5ceb_image.png,0.073346
1259,/kaggle/input/siim-covid19-detection/test/6484...,37bf83df1b86_image,6484393291ea_study,3480,4240,kaggle/tmp/test/37bf83df1b86_image.png,0.008648
1260,/kaggle/input/siim-covid19-detection/test/6484...,b30d2aef985f_image,6484393291ea_study,3480,4240,kaggle/tmp/test/b30d2aef985f_image.png,0.009564
1261,/kaggle/input/siim-covid19-detection/test/531a...,a37a362df0ac_image,531aa20ff7c3_study,3480,4240,kaggle/tmp/test/a37a362df0ac_image.png,0.631295


In [34]:
YOLO_WEIGHTS = '../input/weights-of-yolov5-150-epochs/best.pt'

In [35]:
from shutil import copytree
copytree('/kaggle/input/yolov5-repo/yolov5-master', '/kaggle/tmp/yolov5')

'/kaggle/tmp/yolov5'

In [36]:
from numba import cuda
import torch
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

<weakproxy at 0x7f8f1d4b4170 to Device at 0x7f9555703510>

In [37]:
!python /kaggle/tmp/yolov5/detect.py --weights {YOLO_WEIGHTS}\
--img 512\
--conf 0.001\
--iou 0.5\
--source {TEST_PATH}\
--save-txt --save-conf --exist-ok

detect: weights=['../input/weights-of-yolov5-150-epochs/best.pt'], source=kaggle/tmp/test, imgsz=512, conf_thres=0.001, iou_thres=0.5, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False
requirements: /kaggle/working/requirements.txt not found, check failed.
image 1/1263 /kaggle/working/kaggle/tmp/test/0026720152f5_image.png: 512x512 24 0. opacitys, Done. (0.041s)
image 2/1263 /kaggle/working/kaggle/tmp/test/00fc8fc35dc1_image.png: 512x512 12 0. opacitys, Done. (0.026s)
image 3/1263 /kaggle/working/kaggle/tmp/test/01c3512eebc3_image.png: 512x512 20 0. opacitys, Done. (0.026s)
image 4/1263 /kaggle/working/kaggle/tmp/test/01f948f8e544_image.png: 512x512 29 0. opacitys, Done. (0.025s)
image 5/1263 /kaggle/working/kaggle/tmp/test/022146012034_image.png: 

In [38]:
yolo_pred_path = 'runs/detect/exp/labels'

In [39]:
for i in tqdm(range(test_df.shape[0])):
    image_id = test_df.loc[i, 'image_id']
    original_W, original_H = test_df.loc[i, ['original_W', 'original_H']]
    pred_path = yolo_pred_path + '/' + image_id + '.txt'
    
    bboxes = []
    with open(pred_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            arr = line.strip().split(' ')
            class_id = 'opacity'
            x = float(arr[1])
            y = float(arr[2])
            w = float(arr[3])
            h = float(arr[4])
            x1 = x - (w / 2)  
            x2 = x + (w / 2)    
            y1 = y - (h / 2)
            y2 = y + (h / 2)
            conf = arr[5]

            x1 = int(round(x1 * original_W))
            y1 = int(round(y1 * original_H))
            x2 = int(round(x2 * original_W))
            y2 = int(round(y2 * original_H))

            bboxes.append(f"{class_id} {conf} {x1} {y1} {x2} {y2}")
    string = ' '.join(bboxes)
    #print(string)
    test_df.loc[i, 'PredictionString_yolo'] = string

100%|██████████| 1263/1263 [00:01<00:00, 750.95it/s]


In [40]:
test_df

,filepath,image_id,study_id,original_W,original_H,image_path,none,PredictionString_yolo
0,/kaggle/input/siim-covid19-detection/test/2fb1...,a29c5a68b07b_image,2fb11712bc93_study,2320,2828,kaggle/tmp/test/a29c5a68b07b_image.png,0.018594,opacity 0.00103111 1455 922 2270 2828 opacity ...
1,/kaggle/input/siim-covid19-detection/test/19c6...,9850b5470fd6_image,19c66935e737_study,2330,2382,kaggle/tmp/test/9850b5470fd6_image.png,0.082357,opacity 0.00104978 1525 530 2148 1428 opacity ...
2,/kaggle/input/siim-covid19-detection/test/2fc5...,8d6dea06a032_image,2fc50bf199cd_study,2422,3344,kaggle/tmp/test/8d6dea06a032_image.png,0.103633,opacity 0.00101074 1533 849 2105 2227 opacity ...
3,/kaggle/input/siim-covid19-detection/test/97c5...,dfc5c09a50bc_image,97c5d6eb413d_study,1140,1387,kaggle/tmp/test/dfc5c09a50bc_image.png,0.174045,opacity 0.00107962 728 485 900 675 opacity 0.0...
4,/kaggle/input/siim-covid19-detection/test/593c...,7230234e120a_image,593c3f815635_study,2318,2383,kaggle/tmp/test/7230234e120a_image.png,0.035506,opacity 0.00111306 412 1168 1073 1741 opacity ...
...,...,...,...,...,...,...,...,...
1258,/kaggle/input/siim-covid19-detection/test/b12d...,a43200bd5ceb_image,b12d2e5c39b7_study,2336,2836,kaggle/tmp/test/a43200bd5ceb_image.png,0.073346,opacity 0.00106331 1291 543 1711 1329 opacity ...
1259,/kaggle/input/siim-covid19-detection/test/6484...,37bf83df1b86_image,6484393291ea_study,3480,4240,kaggle/tmp/test/37bf83df1b86_image.png,0.008648,opacity 0.00114178 20 4016 1135 4240 opacity 0...
1260,/kaggle/input/siim-covid19-detection/test/6484...,b30d2aef985f_image,6484393291ea_study,3480,4240,kaggle/tmp/test/b30d2aef985f_image.png,0.009564,opacity 0.00109061 1176 778 1672 1698 opacity ...
1261,/kaggle/input/siim-covid19-detection/test/531a...,a37a362df0ac_image,531aa20ff7c3_study,3480,4240,kaggle/tmp/test/a37a362df0ac_image.png,0.631295,opacity 0.00114888 686 2029 1509 2774 opacity ...


In [41]:
sys.path.append('/kaggle/working/mmdetection')

In [42]:
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

from mmdet.apis import set_random_seed

import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

import mmcv
from mmcv import Config
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmdet.apis import single_gpu_test
from mmdet.datasets import build_dataloader, build_dataset

11.0
GCC 7.3


In [43]:
baseline_cfg_path = '../input/cascade-rnn/job4_cascade_rcnn_x101_32x4d_fpn_1x_coco.py'
cfg = Config.fromfile(baseline_cfg_path)
cfg.classes = ("Covid_Abnormality")
cfg.data.test.img_prefix = ''
cfg.data.test.classes = cfg.classes

for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 1

cfg.seed = 211
set_random_seed(211, deterministic=False)
cfg.gpu_ids = [0]

cfg.data.test.pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip', direction='horizontal'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='DefaultFormatBundle'),
                    dict(type='Collect', keys=['img'])
                ])
        ]

cfg.test_pipeline = [
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip', direction='horizontal'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='DefaultFormatBundle'),
                    dict(type='Collect', keys=['img'])
                ])
        ]


cfg.model.test_cfg.rcnn.score_thr = 0.001

In [44]:
WEIGHTS_FILE = '../input/siimmmdetectioncascadercnnweightbias/epoch_10.pth'

In [45]:
def format_pred(boxes, scores, labels):
    pred_strings = []
    label_str = ['opacity']
    for label, score, bbox in zip(labels, scores, boxes):
        xmin, ymin, xmax, ymax = bbox.astype(np.int64)
        pred_strings.append(f"{label_str[int(label)]} {score:.16f} {xmin} {ymin} {xmax} {ymax}")
    return " ".join(pred_strings)

In [46]:
viz_images = []
results = []
score_threshold = cfg.model.test_cfg.rcnn.score_thr

In [47]:
options = dict(classes = ("Covid_Abnormality"))
model = init_detector(cfg, WEIGHTS_FILE, device='cuda:0')

/kaggle/working/mmdetection/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '


Use load_from_local loader


/kaggle/working/mmdetection/mmdet/apis/inference.py:47: UserWarning: Class names are not saved in the checkpoint's meta data, use COCO classes by default.
  warnings.warn('Class names are not saved in the checkpoint\'s '


In [48]:
with torch.no_grad():
    for i in tqdm(range(test_df.shape[0])):
        original_H = test_df.loc[i, 'original_H']
        original_W = test_df.loc[i, 'original_W']
        path = test_df.loc[i, 'image_path']
        predictions = inference_detector(model, path)
        boxes, scores, labels = (list(), list(), list())

        for k, cls_result in enumerate(predictions):
            if cls_result.size != 0:
                if len(labels)==0:
                    boxes = np.array(cls_result[:, :4])
                    scores = np.array(cls_result[:, 4])
                    labels = np.array([k]*len(cls_result[:, 4]))
                else:    
                    boxes = np.concatenate((boxes, np.array(cls_result[:, :4])))
                    scores = np.concatenate((scores, np.array(cls_result[:, 4])))
                    labels = np.concatenate((labels, [k]*len(cls_result[:, 4])))
                    

        indexes = np.where(scores > score_threshold)
        boxes = boxes[indexes]
        scores = scores[indexes]
        labels = labels[indexes]

        if len(labels) != 0:
            h_ratio = original_H/512
            w_ratio = original_W/512
            boxes[:, [0, 2]] *= w_ratio
            boxes[:, [1, 3]] *= h_ratio

            PredictionString = format_pred(boxes, scores, labels)
            test_df.loc[i, 'PredictionString_crnn'] = PredictionString

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|          | 0/1263 [00:00<?, ?it/s]/kaggle/working/mmdetection/mmdet/core/anchor/anchor_generator.py:323: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/kaggle/working/mmdetection/mmdet/core/anchor/anchor_generator.py:360: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  '``single_level_grid_anchors`` would be deprecated soon. '
100%|██████████| 1263/1263 [02:53<00:00,  7.28it/s]


In [49]:
sys.path.append('/kaggle/input/weightedboxesfusionofficial/ensemble_boxes')
from ensemble_boxes_wbf import weighted_boxes_fusion

In [50]:
for i in tqdm(range(test_df.shape[0])):
    wbf_bboxes = []
    wbf_scores = []
    wbf_labels = []
    scale_value = 10000
    
    yolo_pred = test_df.loc[i, 'PredictionString_yolo']
    yolo_pred = np.array(yolo_pred.split()).reshape(-1, 6)
    
    crnn_pred = test_df.loc[i, 'PredictionString_crnn']
    crnn_pred = np.array(crnn_pred.split()).reshape(-1, 6)
    
    yolo_labels = [1 for x in yolo_pred[:, 0] if x == 'opacity']
    crnn_labels = [1 for x in crnn_pred[:, 0] if x == 'opacity']
    
    yolo_scores = yolo_pred[:, 1]
    crnn_scores = crnn_pred[:, 1]
    
    wbf_bboxes.append(yolo_pred[:, 2:].astype(int) / scale_value)
    wbf_bboxes.append(crnn_pred[:, 2:].astype(int) / scale_value)
    
    wbf_scores.append(yolo_scores.astype(float))
    wbf_scores.append(crnn_scores.astype(float))
    
    wbf_labels.append(yolo_labels)
    wbf_labels.append(crnn_labels)
    
    weights = [0.4, 0.6]
    
    bboxes, confidences, labels = weighted_boxes_fusion(wbf_bboxes, wbf_scores, wbf_labels, weights=weights, 
                                                       iou_thr=0.625, conf_type='avg', allows_overflow=True)
    
    bboxes = np.round(bboxes*scale_value).astype(int)
    pred_string = ''
    for j, conf in enumerate(confidences):
        pred_string += f'opacity {conf} ' + ' '.join(map(str, bboxes[j])) + ' '
    test_df.loc[i, 'PredictionString'] = pred_string + ' none ' + str(test_df.loc[i, 'none']) + ' 0 0 1 1'
    
    
    
    
    

100%|██████████| 1263/1263 [00:10<00:00, 117.97it/s]


In [51]:
image_df = test_df[['image_id', 'PredictionString']]
image_df = image_df.rename(columns={'image_id':'id'})

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [52]:
image_df

,id,PredictionString
0,a29c5a68b07b_image,opacity 0.69740891456604 534 684 1203 2317 opa...
1,9850b5470fd6_image,opacity 0.515137791633606 292 685 991 1928 opa...
2,8d6dea06a032_image,opacity 0.501139223575592 224 1932 738 3049 op...
3,dfc5c09a50bc_image,opacity 0.4846498966217041 288 377 485 792 opa...
4,7230234e120a_image,opacity 0.5313740968704224 1380 1222 2107 1948...
...,...,...
1258,a43200bd5ceb_image,opacity 0.33088213205337524 1249 336 1812 2048...
1259,37bf83df1b86_image,opacity 0.6194295883178711 465 1390 1239 2717 ...
1260,b30d2aef985f_image,opacity 0.595714807510376 2243 1380 3036 2732 ...
1261,a37a362df0ac_image,opacity 0.3095042109489441 486 1557 1307 2717 ...


In [53]:
#image_df = test_df[['image_id', 'PredictionString']].rename(columns={'image_id': 'id'}, inplace=True)
study_df = study_df.drop_duplicates(subset=['id'])
study_df = study_df.drop(columns=['negative', 'typical', 'indeterminate', 'atypical'])

df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
for i in tqdm(range(df.shape[0])):
    level = df.loc[i, 'id'].split('_')[1]
    im_id = df.loc[i, 'id']
    if level == 'study':
        df.loc[i, 'PredictionString'] = study_df[study_df.id==im_id]['PredictionString'].values[0]
    elif level == 'image':
        df.loc[i, 'PredictionString'] = image_df[image_df.id==im_id]['PredictionString'].values[0]
df.to_csv('/kaggle/working/submission.csv', index=False)

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 2477/2477 [00:02<00:00, 1110.90it/s]


In [54]:
import shutil
shutil.rmtree('/kaggle/tmp/')
shutil.rmtree('/kaggle/working/gdcm')
shutil.rmtree('/kaggle/working/mmdetection')
shutil.rmtree('/kaggle/working/runs')
shutil.rmtree('/kaggle/working/kaggle')